* 읽은 문서 파일 내 테이블에 해당하는 부분을 마크다운으로 변경하는 기능 확인

In [4]:
from docx import Document

def read_tables_from_docx(file_path):
    doc = Document(file_path)
    tables_data = []
    
    for table in doc.tables:
        table_content = []
        for row in table.rows:
            row_content = [cell.text.strip() for cell in row.cells]
            table_content.append(row_content)
        tables_data.append(table_content)
    
    return doc, tables_data


In [5]:
# 표를 텍스트 형식으로 변환 (마크다운 형태)
def convert_table_to_markdown(table):
    if not table:
        return ""
    
    markdown = []
    headers = table[0]  # 첫 번째 행을 헤더로 사용
    markdown.append("| " + " | ".join(headers) + " |")
    markdown.append("| " + " | ".join(["---"] * len(headers)) + " |")
    
    for row in table[1:]:
        markdown.append("| " + " | ".join(row) + " |")
    
    return "\n".join(markdown)

def replace_tables_with_markdown(original_doc, tables_data):
    """
    기존 문서의 표 데이터를 마크다운으로 교체
    """
    new_doc = Document()
    for element in original_doc.element.body:
        if element.tag.endswith('tbl'):  # 표 요소인지 확인
            if tables_data:
                # 표 데이터를 마크다운 형식으로 변환
                table = tables_data.pop(0)
                markdown = convert_table_to_markdown(table)
                # 문서에 마크다운 추가
                new_doc.add_paragraph(markdown)
        else:
            # 표가 아닌 기존 문서 내용을 그대로 복사
            new_doc.add_paragraph(element.text)
    return new_doc

def save_docx_with_markdown(input_file, output_file):
    """
    기존 문서의 내용을 유지하면서 표를 마크다운으로 변환하여 새 문서로 저장
    """
    # 원본 문서 및 표 데이터 읽기
    original_doc, tables_data = read_tables_from_docx(input_file)

    # 표를 마크다운으로 교체한 문서 생성
    updated_doc = replace_tables_with_markdown(original_doc, tables_data)

    # 새 문서 저장
    updated_doc.save(output_file)


In [6]:
# 실행 코드
input_file = "GA4_schema.docx"  # 입력 파일 경로
output_file = "GA4_schema_markdown.docx"  # 출력 파일 경로

save_docx_with_markdown(input_file, output_file)